In [289]:
import polars as pl
from datetime import datetime
from typing import Tuple, List
import warnings
import sys
import numpy as np
import scipy
import statsmodels.api as sm
from alpha101_prod import CalcAlpha101Factor

POS_RET_PCT_SCALE_THRESHOLD = 0.01

input_path = "data/all_data_1d.parquet"
output_path = "data/predictions.parquet"
is_prod = False

In [290]:
def AddPastReturnFactor(input_df: pl.DataFrame, day_num: int) -> pl.DataFrame:
    input_df = input_df.sort(["symbol", "open_time"])
    # 千万注意，这里是计算此时刻相对于前一时刻的return，不能使用未来信息 -> shift(1)是整体往下移动
    for i in range(1, day_num + 1):
        input_df = input_df.with_columns(
            ((pl.col("close") / pl.col("close").shift(i) - 1) * 100)
            .over("symbol")  # Applying the function over each symbol group
            .alias(f"past_{i}day_close_return")
        )

    # 默认使用过去1天的return作为return列
    input_df = input_df.with_columns(pl.col("past_1day_close_return").alias("return"))

    return input_df

In [291]:
def CalcDayPositionScale(
    input_df: pl.DataFrame, day_num: int, trade_long_rank: int, trade_short_rank: int
) -> pl.DataFrame:
    # print(input_df["linear_compound_factor_1day"])
    agg_avg_ret_list = []
    for i in range(1, day_num + 1):
        for side in ["long", "short"]:
            sort_desc = True if side == "long" else False
            trade_rank_num = trade_long_rank if side == "long" else trade_short_rank

            input_df = input_df.with_columns(
                pl.col(f"linear_compound_factor_{i}day")
                .rank(descending=sort_desc)
                .over("open_time")
                .alias(f"symbol_rank_{side}_{i}day")
            ).with_columns(
                pl.when(pl.col(f"symbol_rank_{side}_{i}day") <= trade_rank_num)
                .then(pl.col(f"close_price_fut_{i}day_ret"))
                .otherwise(None)
                .alias(f"total_{side}_value_scale_{i}day")
            )

            cur_agg_avg_ret = (
                input_df.group_by("open_time")
                .agg(
                    pl.col(f"total_{side}_value_scale_{i}day")
                    .mean()
                    .alias(f"fut_mean_{side}_ret_{i}day"),
                )
                .sort("open_time")
            )
            agg_avg_ret_list.append(cur_agg_avg_ret)

    agg_avg_ret_df = agg_avg_ret_list[0]
    for df in agg_avg_ret_list[1:]:
        agg_avg_ret_df = agg_avg_ret_df.join(df, on="open_time", how="right")

    # Sort the final DataFrame by open_time
    agg_avg_ret_df = agg_avg_ret_df.sort("open_time")
    agg_avg_ret_df = agg_avg_ret_df.select(
        pl.col(
            ["open_time"]
            + [col for col in agg_avg_ret_df.columns if col != "open_time"]
        )
    )

    for i in range(1, day_num + 1):
        for side in ["long", "short"]:
            agg_avg_ret_df = agg_avg_ret_df.with_columns(
                pl.col(f"fut_mean_{side}_ret_{i}day")
                .shift(i)
                .alias(f"past_mean_{side}_ret_{i}day"),
            )

    for i in range(1, day_num + 1):
        for side in ["long", "short"]:
            bullish_scale = 1.2 if side == "long" else 0.8
            bearish_scale = 0.8 if side == "long" else 1.2
            agg_avg_ret_df = agg_avg_ret_df.with_columns(
                pl.when(
                    (pl.col(f"past_mean_long_ret_{i}day") > POS_RET_PCT_SCALE_THRESHOLD)
                    & (
                        pl.col(f"past_mean_short_ret_{i}day")
                        > POS_RET_PCT_SCALE_THRESHOLD
                    )
                )
                .then(bullish_scale)
                .when(
                    (pl.col(f"past_mean_long_ret_{i}day") < POS_RET_PCT_SCALE_THRESHOLD)
                    & (
                        pl.col(f"past_mean_short_ret_{i}day")
                        < POS_RET_PCT_SCALE_THRESHOLD
                    )
                )
                .then(bearish_scale)
                .otherwise(1.0)
                .alias(f"{side}_value_scale_{i}day")
            )
    return input_df, agg_avg_ret_df


In [292]:
def AddTotalPosValueScale(
    input_df: pl.DataFrame, day_num: int, trade_long_rank: int, trade_short_rank: int
) -> pl.DataFrame:
    _, day_scale_df = CalcDayPositionScale(
        input_df, day_num, trade_long_rank, trade_short_rank
    )

    # only need the scale columns
    select_col = [col for col in day_scale_df.columns if "scale" in col]
    day_scale_df = day_scale_df.select(pl.col(["open_time"] + select_col))

    # print(f"before join: {input_df}")
    # print(f"day scale df: ", day_scale_df)

    input_df = input_df.join(day_scale_df, on="open_time", how="left")

    # print(f"after join     : {input_df}")
    return input_df, day_scale_df


In [293]:
def AddAmihud(input_df: pl.DataFrame, window_size: int = 10) -> pl.DataFrame:
    # Calculate rolling sums for absolute returns and quote volume
    input_df = input_df.with_columns(
        pl.col("return")
        .abs()
        .rolling_sum(window_size=window_size)
        .over("symbol")
        .alias("rolling_abs_return_sum"),
        pl.col("quote_volume")
        .rolling_sum(window_size=window_size)
        .over("symbol")
        .alias("rolling_quote_volume_sum"),
    )

    # Calculate Amihud illiquidity measure
    input_df = input_df.with_columns(
        (pl.col("rolling_abs_return_sum") / pl.col("rolling_quote_volume_sum"))
        .over("symbol")  # Apply the final operation within each symbol group
        .alias("amihud")
    )

    # Drop intermediate columns
    input_df = input_df.drop(["rolling_abs_return_sum", "rolling_quote_volume_sum"])

    return input_df

In [294]:
def AddFutureRetCol(input_df: pl.DataFrame, day_num: int) -> pl.DataFrame:
    input_df = input_df.sort(["symbol", "open_time"])
    for i in range(1, day_num + 1):
        input_df = input_df.with_columns(
            ((pl.col("close").shift(-i) / pl.col("close") - 1) * 100)
            .over("symbol")  # Applying the function over each symbol group
            .alias(f"close_price_fut_{i}day_ret")
        )
        input_df = input_df.with_columns(
            ((pl.col("open").shift(-i) / pl.col("open") - 1) * 100)
            .over("symbol")  # Applying the function over each symbol group
            .alias(f"open_price_fut_{i}day_ret")
        )
    return input_df

In [295]:
def fama_macbeth_get_factor_weight(
    train_data: pl.DataFrame,
    update_pos_days: int,
    factor_num: int,
    factor_combination_list: List[str],
) -> Tuple[np.ndarray, float]:
    # Drop rows containing any null values
    train_data = train_data.drop_nulls()

    total_weights_sum = np.zeros(factor_num)
    unique_times = train_data.select(pl.col("open_time").sort()).unique().to_numpy()

    constant_sum = 0.0

    for each_time in unique_times:
        y_column_name = f"close_price_fut_{update_pos_days}day_ret"
        assert (
            y_column_name in train_data.columns
        ), f"Column {y_column_name} (as y) not found in train data"

        slice_data = train_data.filter(pl.col("open_time") == each_time).fill_nan(0)

        X = slice_data[factor_combination_list].to_numpy()
        X = sm.add_constant(X)  # Add constant term (intercept)
        y = slice_data[y_column_name].to_numpy()

        model = sm.OLS(y, X)
        results = model.fit()
        weights = results.params[1:]
        constant_sum += results.params[0]  # constant term

        while weights.shape[0] < total_weights_sum.shape[0]:
            weights = np.append(weights, 0)

        total_weights_sum += weights

    total_weights_sum /= len(unique_times)
    avg_const_term = constant_sum / len(unique_times)

    return total_weights_sum, avg_const_term


In [296]:
def CalcLinearCompoundFactor(
    input_df: pl.DataFrame,
    day_num: int,
    factor_combination_list: list,
    date_threshold: datetime = datetime(2023, 1, 1),
) -> pl.DataFrame:

    factor_num = len(factor_combination_list)

    for cur_update_position_time in range(1, day_num + 1):
        cur_fut_ret_column_name = f"close_price_fut_{cur_update_position_time}day_ret"

        non_nan_result = input_df.filter(
            (pl.col(cur_fut_ret_column_name).is_not_nan())
            & (pl.col(cur_fut_ret_column_name).is_not_null())
        ).sort(["open_time", "symbol"])

        non_nan_linear_x = non_nan_result.select(
            ["open_time", "symbol", cur_fut_ret_column_name] + factor_combination_list
        )

        date_threshold_ms = date_threshold.timestamp() * 1000
        linear_x_train = non_nan_linear_x.filter(
            pl.col("open_time") < date_threshold_ms
        )

        # # 检查过确实只使用训练数据拟合，没有用测试信息
        # print(linear_x_train.sort(by="open_time"))

        weighted_factors, const_term = fama_macbeth_get_factor_weight(
            linear_x_train,
            cur_update_position_time,
            factor_num=factor_num,
            factor_combination_list=factor_combination_list,
        )

        weighted_sum_expr = pl.lit(const_term)
        for factor, weight in zip(factor_combination_list, weighted_factors):
            weighted_sum_expr += pl.col(factor) * weight

        # 这个代码用于实际交易的时候，我们只需要使用权重计算未来收益率
        input_df = input_df.with_columns(
            weighted_sum_expr.alias(
                f"linear_compound_factor_{cur_update_position_time}day"
            )
        )

    return input_df.filter(
        pl.col("open_time") >= date_threshold_ms
    )  # only return the data after the threshold


In [297]:
def AddVolatilityCol(input_df: pl.DataFrame) -> pl.DataFrame:
    def calculate_volatility(group, bar_name: str, window_size: int = 30):
        return group.with_columns(
            [
                pl.col(bar_name)
                .pct_change()
                .rolling_std(window_size=window_size)
                .alias(f"{bar_name}_price_volatility")
            ]
        )

    input_df = input_df.group_by("symbol").map_groups(
        lambda x: calculate_volatility(x, "open", window_size=30)
    )
    input_df = input_df.group_by("symbol").map_groups(
        lambda x: calculate_volatility(x, "close", window_size=30)
    )
    return input_df

In [298]:
FACTOR_COMBINATION_LIST = [
    "amihud",
    "alpha30",
    "alpha36",
    "alpha45",
    "alpha40",
]
UPDATE_POSITION_TIME = 10

input_data = pl.read_parquet(input_path)
input_data = input_data.with_columns(pl.from_epoch(pl.col("open_time"), time_unit="ms").cast(pl.Datetime('ms')).alias("open_time"))

# for production need
input_data = input_data.filter(
    ~pl.col("symbol").is_in(
        ["BTCUSDT", "ETHUSDT", "BCHUSDT", "LTCUSDT", "ETCUSDT", "LINKUSDT", "AVAXUSDT", "SOLUSDT"]
    )
).filter(pl.col("symbol").str.ends_with("USDT"))

input_data = AddPastReturnFactor(input_data, day_num=10)
input_data = AddVolatilityCol(input_data)
input_data = AddAmihud(input_data)

# Calculate alpha101 factor
alpha101_factor_list = [x for x in FACTOR_COMBINATION_LIST if "alpha" in x]
input_data = CalcAlpha101Factor(input_data, calc_factor_list=alpha101_factor_list)

input_data

alpha30
alpha36
alpha45
alpha40


symbol,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,past_1day_close_return,past_2day_close_return,past_3day_close_return,past_4day_close_return,past_5day_close_return,past_6day_close_return,past_7day_close_return,past_8day_close_return,past_9day_close_return,past_10day_close_return,return,open_price_volatility,close_price_volatility,amihud,alpha30,alpha36,alpha45,alpha40
str,datetime[ms],f64,f64,f64,f64,f64,i64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""SXPUSDT""",2021-01-01 00:00:00,0.7414,0.7719,0.6767,0.7136,3.9652e7,1609545599999,2.8827e7,149686,1.8155e7,1.3207e7,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0
"""SXPUSDT""",2021-01-02 00:00:00,0.7137,0.763,0.6873,0.7486,3.6906829e7,1609631999999,2.6761e7,141108,1.7186e7,1.2469e7,4.904709,null,null,null,null,null,null,null,null,null,4.904709,null,null,null,0.0,0.0,0.0,0.0
"""SXPUSDT""",2021-01-03 00:00:00,0.7488,0.8345,0.7375,0.7749,9.4022e7,1609718399999,7.3114e7,299749,4.3723e7,3.3999e7,3.513225,8.590247,null,null,null,null,null,null,null,null,3.513225,null,null,null,0.0,0.0,0.0,0.0
"""SXPUSDT""",2021-01-04 00:00:00,0.7748,0.9231,0.6742,0.7327,1.16957542e8,1609804799999,9.0092e7,367433,5.3101e7,4.0962e7,-5.445864,-2.123965,2.67657,null,null,null,null,null,null,null,-5.445864,null,null,null,0.0,0.0,0.0,0.0
"""SXPUSDT""",2021-01-05 00:00:00,0.7327,0.78,0.6906,0.7573,6.3964e7,1609891199999,4.7417e7,200282,2.9176e7,2.1648e7,3.357445,-2.271261,1.162169,6.123879,null,null,null,null,null,null,3.357445,null,null,null,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""CAKEUSDT""",2024-10-30 00:00:00,1.8409,1.8433,1.7927,1.8232,3.38621e6,1730332799999,6.1668e6,66819,1.557219e6,2.8367e6,-0.977623,1.120355,2.819761,3.543844,3.490946,-3.257986,-1.368677,-3.890353,-4.937692,-8.095574,-0.977623,0.030578,0.029237,2.5984e-7,0.066818,2.590627,-0.370358,-0.571105
"""CAKEUSDT""",2024-10-31 00:00:00,1.8231,1.8258,1.7424,1.7621,3.237795e6,1730419199999,5.7607e6,62325,1.506145e6,2.6803e6,-3.351251,-4.296111,-2.268441,-0.625987,0.07383,0.022705,-6.500053,-4.67406,-7.111228,-8.123468,-3.351251,0.029191,0.027464,2.9391e-7,0.111706,2.609187,-0.216572,-0.473696
"""CAKEUSDT""",2024-11-01 00:00:00,1.7619,1.7904,1.7073,1.7257,3.671142e6,1730505599999,6.4139e6,69595,1.767105e6,3.0879e6,-2.065717,-5.34774,-6.273083,-4.287299,-2.678773,-1.993412,-2.043481,-8.431497,-6.643224,-9.030047,-2.065717,0.027406,0.027177,3.1974e-7,0.132742,2.462033,0.37967,-0.456759


In [299]:
# 截面
input_data = input_data.sort(by=["symbol", "open_time"])
original_columns = input_data.columns
for c in FACTOR_COMBINATION_LIST:
    print(c)
    input_data = input_data.with_columns(pl.col(c).mean().over("open_time").alias("mean_" + c))
    input_data = input_data.with_columns(pl.col(c).std().over("open_time").alias("std_" + c))
    input_data = input_data.with_columns(((pl.col(c) - pl.col("mean_" + c)) / pl.col("std_" + c)).alias(c))
input_data = input_data.select(original_columns)
input_data


amihud
alpha30
alpha36
alpha45
alpha40


symbol,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,past_1day_close_return,past_2day_close_return,past_3day_close_return,past_4day_close_return,past_5day_close_return,past_6day_close_return,past_7day_close_return,past_8day_close_return,past_9day_close_return,past_10day_close_return,return,open_price_volatility,close_price_volatility,amihud,alpha30,alpha36,alpha45,alpha40
str,datetime[ms],f64,f64,f64,f64,f64,i64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1000BONKUSDT""",2023-11-22 00:00:00,0.00478,0.004825,0.004076,0.004531,1.2701e10,1700697599999,5.6368e7,320715,6.1849e9,2.7467e7,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-1.581977,-1.532579,-0.643618,1.365114
"""1000BONKUSDT""",2023-11-23 00:00:00,0.004531,0.004858,0.00393,0.004267,1.8971e10,1700783999999,8.1590e7,573386,8.8982e9,3.8318e7,-5.826528,null,null,null,null,null,null,null,null,null,-5.826528,null,null,null,-1.11916,-1.580292,0.132825,1.422705
"""1000BONKUSDT""",2023-11-24 00:00:00,0.004267,0.004335,0.003835,0.00414,1.7169e10,1700870399999,6.9929e7,475254,7.9410e9,3.2393e7,-2.97633,-8.629442,null,null,null,null,null,null,null,null,-2.97633,null,null,null,-1.11713,-1.543047,0.25099,1.396197
"""1000BONKUSDT""",2023-11-25 00:00:00,0.004143,0.0046,0.004084,0.00434,1.3093e10,1700956799999,5.6988e7,501832,6.3891e9,2.7819e7,4.830918,1.710804,-4.215405,null,null,null,null,null,null,null,4.830918,null,null,null,-1.130631,-1.588192,0.011231,1.301208
"""1000BONKUSDT""",2023-11-26 00:00:00,0.004341,0.004473,0.0039,0.004039,8.2273e9,1701043199999,3.4347e7,344203,3.8247e9,1.5980e7,-6.935484,-2.439614,-5.343333,-10.85853,null,null,null,null,null,null,-6.935484,null,null,null,-1.133583,-1.578087,0.123405,1.004191
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZRXUSDT""",2024-10-30 00:00:00,0.337,0.3454,0.3259,0.3387,3.3094e7,1730332799999,1.1187e7,69215,1.6184e7,5.4729e6,0.534283,5.645664,5.382701,7.149636,6.075791,-0.877963,0.147842,-3.504274,-5.311714,-4.64527,0.534283,0.037009,0.03447,-0.590159,-0.326927,0.302397,-0.395288,-0.697516
"""ZRXUSDT""",2024-10-31 00:00:00,0.3388,0.3402,0.315,0.3175,2.6379e7,1730419199999,8.6137e6,59773,1.2525e7,4.0909e6,-6.259226,-5.758385,-0.966937,-1.213441,0.442898,-0.563733,-7.082236,-6.120639,-9.54416,-11.238468,-6.259226,0.034517,0.032805,-0.413481,-1.31084,0.393852,-1.250315,-0.181491
"""ZRXUSDT""",2024-11-01 00:00:00,0.3176,0.3248,0.3059,0.3109,2.1759e7,1730505599999,6.8310e6,54349,1.0382e7,3.2587e6,-2.07874,-8.207854,-7.717424,-3.025577,-3.266957,-1.645049,-2.630755,-9.013755,-8.072147,-11.424501,-2.07874,0.032877,0.032974,-0.380879,-1.074798,0.342494,-1.388746,-0.367365


In [300]:
# below for combine factors
print(f"begin to calc linear compound factor: {FACTOR_COMBINATION_LIST}")
input_data = AddFutureRetCol(input_data, UPDATE_POSITION_TIME)
input_data

input_data = CalcLinearCompoundFactor(
    input_data, UPDATE_POSITION_TIME, FACTOR_COMBINATION_LIST
)

input_data, day_scale_df = AddTotalPosValueScale(
    input_data, day_num=10, trade_long_rank=20, trade_short_rank=10
)

if is_prod:
    # for normal run, save all data
    results = input_data
else:
    # for backtest and research, remove the last few rows and symbols whose min value is larger then 5 USDT
    results = input_data.filter(pl.col("close_price_fut_7day_ret").is_not_null())
results.write_parquet(output_path)
results

begin to calc linear compound factor: ['amihud', 'alpha30', 'alpha36', 'alpha45', 'alpha40']


symbol,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,past_1day_close_return,past_2day_close_return,past_3day_close_return,past_4day_close_return,past_5day_close_return,past_6day_close_return,past_7day_close_return,past_8day_close_return,past_9day_close_return,past_10day_close_return,return,open_price_volatility,close_price_volatility,amihud,alpha30,alpha36,alpha45,alpha40,close_price_fut_1day_ret,open_price_fut_1day_ret,close_price_fut_2day_ret,open_price_fut_2day_ret,close_price_fut_3day_ret,open_price_fut_3day_ret,close_price_fut_4day_ret,…,open_price_fut_7day_ret,close_price_fut_8day_ret,open_price_fut_8day_ret,close_price_fut_9day_ret,open_price_fut_9day_ret,close_price_fut_10day_ret,open_price_fut_10day_ret,linear_compound_factor_1day,linear_compound_factor_2day,linear_compound_factor_3day,linear_compound_factor_4day,linear_compound_factor_5day,linear_compound_factor_6day,linear_compound_factor_7day,linear_compound_factor_8day,linear_compound_factor_9day,linear_compound_factor_10day,long_value_scale_1day,short_value_scale_1day,long_value_scale_2day,short_value_scale_2day,long_value_scale_3day,short_value_scale_3day,long_value_scale_4day,short_value_scale_4day,long_value_scale_5day,short_value_scale_5day,long_value_scale_6day,short_value_scale_6day,long_value_scale_7day,short_value_scale_7day,long_value_scale_8day,short_value_scale_8day,long_value_scale_9day,short_value_scale_9day,long_value_scale_10day,short_value_scale_10day
str,datetime[ms],f64,f64,f64,f64,f64,i64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1000BONKUSDT""",2023-11-22 00:00:00,0.00478,0.004825,0.004076,0.004531,1.2701e10,1700697599999,5.6368e7,320715,6.1849e9,2.7467e7,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-1.581977,-1.532579,-0.643618,1.365114,-5.826528,-5.209205,-8.629442,-10.732218,-4.215405,-13.32636,-10.85853,…,-13.493724,-14.257338,-14.60251,-11.013021,-18.74477,7.128669,-15.627615,null,null,null,null,null,null,null,null,null,null,1.2,0.8,0.8,1.2,0.8,1.2,1.0,1.0,0.8,1.2,1.0,1.0,0.8,1.2,1.0,1.0,1.2,0.8,1.0,1.0
"""1000BONKUSDT""",2023-11-23 00:00:00,0.004531,0.004858,0.00393,0.004267,1.8971e10,1700783999999,8.1590e7,573386,8.8982e9,3.8318e7,-5.826528,null,null,null,null,null,null,null,null,null,-5.826528,null,null,null,-1.11916,-1.580292,0.132825,1.422705,-2.97633,-5.826528,1.710804,-8.563231,-5.343333,-4.193335,-11.131943,…,-9.909512,-5.507382,-14.279409,13.756738,-10.990951,24.67776,7.062459,null,null,null,null,null,null,null,null,null,null,1.2,0.8,1.2,0.8,1.0,1.0,1.0,1.0,1.2,0.8,1.0,1.0,1.0,1.0,0.8,1.2,1.0,1.0,1.2,0.8
"""1000BONKUSDT""",2023-11-24 00:00:00,0.004267,0.004335,0.003835,0.00414,1.7169e10,1700870399999,6.9929e7,475254,7.9410e9,3.2393e7,-2.97633,-8.629442,null,null,null,null,null,null,null,null,-2.97633,null,null,null,-1.11713,-1.543047,0.25099,1.396197,4.830918,-2.906023,-2.439614,1.73424,-8.405797,-5.296461,-0.193237,…,-8.975861,17.246377,-5.483947,28.502415,13.686431,51.884058,24.724631,null,null,null,null,null,null,null,null,null,null,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.0,1.0,1.2,0.8,1.0,1.0,1.2,0.8
"""1000BONKUSDT""",2023-11-25 00:00:00,0.004143,0.0046,0.004084,0.00434,1.3093e10,1700956799999,5.6988e7,501832,6.3891e9,2.7819e7,4.830918,1.710804,-4.215405,null,null,null,null,null,null,null,4.830918,null,null,null,-1.130631,-1.588192,0.011231,1.301208,-6.935484,4.779146,-12.626728,-2.461984,-4.792627,-8.472122,-5.967742,…,-2.655081,22.580645,17.089066,44.884793,28.457639,74.239631,51.774077,null,null,null,null,null,null,null,null,null,null,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.0,1.0
"""1000BONKUSDT""",2023-11-26 00:00:00,0.004341,0.004473,0.0039,0.004039,8.2273e9,17010431999